
# Iterators

https://docs.python.org/3/tutorial/classes.html#iterators

We've already seen the keyword iteration, referring to constructs like for- and while-loops. Colloquially, an iterable is an object that can be "for-looped" over, and an iterator is the companion object which does so. In this lecture, we'll look into the operation of for-loops in a bit more detail, and reimplement looping over lists. In the next lecture, we'll define custom iterators with novel behavior.

We've already seen several containers: objects that hold other objects. Examples of containers include lists, tuples, sets, and dictionaries. Most default containers already support iteration:

In [1]:
L = [1, 2, 3]
for element in L:
    print(element)

# behind the scenes:
# 1. call `iter(L)`` to get some iterator object, let's call it tmp_iterator_object
# 2. call `next(tmp_iterator_object)` to access element one at a time
# 3. stop once the Exception StopIteration is raised

# iterators are something you can call `__next__()` on
# L can be for-looped over if 
# 1. L has `__iter__()` implemented so that iter(L) returns an iterator
# 2. iterator returned by iter(L) - let's call it LIt - has __next__ implemented

1
2
3


In [2]:
tmp_iterator_object = iter(L)
print(tmp_iterator_object)

In [3]:
while True:
    print(next(tmp_iterator_object))

1
2
3


StopIteration: 

For `L` to be iterable ('for-loop-able'), 

`iter(L)` needs to return an interator object <=> type of `L` has `__iter__()` method that returns an iterator object

let's call that `tmp_iterator_object`,

`next(tmp_iterator_object)` needs to return one element of `L` at a time <==> type of `tmp_iterator_object` has `__next__()` method that returns one element of `L` at a time

`__iter__()`: This method returns the **iterator object** itself. It is called on the initialization of an iterator. This method is also what makes an object iterable.

`__next__()`: This method returns **the next item** from the sequence. On reaching the end of the sequence, it **should raise a StopIteration exception**.



However, not every container-like object automatically supports iteration. For example, here's a simple class that holds a list and...doesn't do much else.


In [6]:
class BoringList:
    
    def __init__(self, L):
        self.L = L
        
    def __str__(self):
        return(str(self.L))

In [7]:
boringL = BoringList([1, 2, 3])
print(boringL)

[1, 2, 3]


In [8]:
for i in boringL: # TypeError: 'BoringList' object is not iterable
    print(i)

TypeError: 'BoringList' object is not iterable

In [19]:
class BoringList:
    
    def __init__(self, L):
        self.L = L
        
    def __str__(self):
        return(str(self.L))
    
    def __iter__(self):
        print("BoringList.__iter__() called")
        return iter(self.L)

In [20]:
boringL = BoringList([1, 2, 3])
print(boringL) # print by __str__() method
for i in boringL: # loop by __iter__() method
    print(i)

[1, 2, 3]
BoringList.__iter__() called
1
2
3




So, what is needed to iterate over a container? 

Briefly, we need to define **an iterator class** that implements a `__next__()` method. <br>
To get the iterator from the container, we need to define an `__iter__()` method that tells Python how to construct an appropriate iterator, with the required `__next__()` method. <br>
If our class itself has a `__next__()` method, then `__iter__()` can just return the object itself. <br>

`myClass.__iter__()` returns an object of class myIterator. <br>
`myIterator.__next__()` returns objects until a StopIteration exception is raised.

Let's try making our `BoringList` iterable. We need an `__iter__()` method and a `__next__()` method. To make the workings of the code transparent, we'll add a few print statements.


In [5]:
class BoringList:
    def __init__(self, L):
        self.L = L
    
    def __str__(self):
        return(str(self.L))
    
    def __iter__(self):
        print("BoringList.__iter__() called")
        return(BoringListIterator(self)) 
    # pass self so that iterator can have access to all of self's stuff
    
# write a iter() method by yourself
class BoringListIterator:
    
    def __init__(self, bL):
        print("BoringListIterator object created")
        self.L = bL.L
        self.i = 0 # need to keep track of current index
        
    def __next__(self):
        print("BoringListItator.__next__() called")
        
        if self.i < len(self.L):
            element = self.L[self.i]
            self.i += 1
            return element
        else:
            print('StopIteration')
            raise StopIteration

#   def __next__(self):
#         if self.i >= len(self.L):
#             raise StopIteration
        
#         self.i += 1
#         return(self.L[self.i-1])

In [6]:
boringL = BoringList([1, 2, 3])
print(boringL)
for i in boringL:
    print(i)

[1, 2, 3]
BoringList.__iter__() called
BoringListIterator object created
BoringListItator.__next__() called
1
BoringListItator.__next__() called
2
BoringListItator.__next__() called
3
BoringListItator.__next__() called
StopIteration


Behind the scenes, the `for` keyword calls `__iter__()` on `B`, creating an object of type `BoringListIterator`. Because this object has a `__next__()` method, the `for`-loop can then operate as expected. Here's the "slow-motion" version:

In [7]:
boringLI = iter(boringL) # BoringListIterator object created

BoringList.__iter__() called
BoringListIterator object created


In [8]:
# repeated until the StopIteration exception is raised. 
next(boringLI)

BoringListItator.__next__() called


1

In [9]:
next(boringLI)

BoringListItator.__next__() called


2

In [10]:
next(boringLI)

BoringListItator.__next__() called


3

Another approach to the same problem is to define a `__next__()` method for `BoringList` itself.

In this case, the `__iter__()` method can simply return `self`.

The `__iter__()` method should reset i to 0 each time to ensure that we always loop starting from the beginning.

In [11]:
class boringList:
    def __init__(self, L):
        self.L = L
        self.i = 0
        
    def __str__(self):
        return(str(self.L))
    
    def __iter__(self):
        self.i = 0
        return(self)
    
    def __next__(self):
        if self.i >= len(self.L):
            raise StopIteration
        
        self.i += 1
        return(self.L[self.i-1])


> **Why no `__next__()` is called in the `__iter__()`?**
>
> Your confusion might stem from not seeing an explicit call to `__next__()` within the `__iter__()` method or the for-loop in your code. However, the call to `__next__()` is made automatically by Python when iterating over an object in a for-loop, not something you need to explicitly write in your `__iter__()` method or when using the loop.

> **Why there is a duplicate line `self.i = 0` in the `__iter__()?`**
> 
> Including self.i = 0 within the `__iter__()` method, even though self.i is initially set to 0 in the `__init__()` method, serves a specific purpose in the context of making the object iterable and allowing it to be iterated over multiple times.
> 
> When an object is first initialized, setting self.i = 0 in `__init__()` establishes the starting point for iteration. However, once you iterate over the object once, self.i will have been incremented with each call to `__next__()` to the point where it reaches the end of the list, causing a StopIteration exception to be raised and signaling the end of the iteration.



In [20]:
# now this again works
boringL = BoringList([1, 2, 3])
for w in boringL:
    print(w)

BoringList.__iter__() called
BoringListIterator object created
BoringListItator.__next__() called
1
BoringListItator.__next__() called
2
BoringListItator.__next__() called
3
BoringListItator.__next__() called
StopIteration
